In [1]:
%pylab inline
import sys
sys.path.append('../build_gcc/')
import versor as vsr

Populating the interactive namespace from numpy and matplotlib


ImportError: No module named versor_pybind11

In [ ]:
np.set_printoptions(precision=2, suppress=True)

# Hand Eye Calibration

In [ ]:
M_object_in_world = vsr.Trs.from_vector([1,1,0.1]) * vsr.Rot.from_bivector(vsr.Biv(1,0,0) * np.pi/4.0)
print(M_object_in_world)

In [ ]:
log = M_object_in_world.log()
log

In [ ]:
M_eye_in_hand = vsr.Trs.from_vector([0.1,0.2,0.3]) * vsr.Rot.from_bivector(vsr.Biv(0,1,0) * np.pi/6.)
print(M_eye_in_hand)

In [ ]:
M_hand_in_world = vsr.Trs.from_vector([0.0,0.0,1.0]) * vsr.Rot.from_bivector(vsr.Biv(1,1,1).unit() * np.pi/6.)
M_hand_in_world

In [ ]:
M_object_in_eye = M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world
print(M_object_in_eye.to_array())

In [ ]:
print(M_hand_in_world)

In [ ]:
print(M_hand_in_world.log().mot())

In [ ]:
pose_pairs = np.array([(M_hand_in_world,
               M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world) 
              for M_hand_in_world in [vsr.Trs.from_vector(vsr.Vec(*np.random.rand(3))) *
                                      vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
                                                            np.random.rand() * np.pi)
                                      for i in range(4)]])

In [ ]:
M_object_in_world = vsr.Trs.from_vector([1,1,0.1]) * vsr.Rot.from_bivector(vsr.Biv(1,0,0) * np.pi/4.0)
M_eye_in_hand = vsr.Trs.from_vector([-0.5,0.2,-0.3]) * vsr.Rot.from_bivector(vsr.Biv(1,1,1).unit() * np.pi/6.)
pose_pairs = np.array([(M_hand_in_world,
               (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
              for M_hand_in_world in [vsr.Trs.from_vector(vsr.Vec(*np.random.rand(3))) *
                                      vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
                                                            np.random.rand() * np.pi)
                                      for i in range(4)]])
n = len(pose_pairs)
As = pose_pairs[:,0]
Bs = pose_pairs[:,1]
LAs = []
LBs = []
MAs = []
MBs = []
for i in range(n):
    for j in range(i+1,n):
        MBs.append((Bs[j] * Bs[i].rev()))     
        MAs.append((As[j].rev() * As[i]))
        LBs.append((Bs[j] * Bs[i].rev()).log())     
        LAs.append((As[j].rev() * As[i]).log())
print(len(LAs))

In [ ]:
print LBs[1].unit().spin(M_eye_in_hand)
print(LAs[1].unit())

In [ ]:
[np.allclose(M_object_in_world.to_array(),
             (M_hand_in_world * M_eye_in_hand * M_object_in_eye).to_array())
             for (M_hand_in_world, M_object_in_eye) in pose_pairs]

In [ ]:
[np.allclose(a[0],b[0]) for a,b in zip(MAs, MBs)]

In [ ]:
def skew(v):
    skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
    return skv - skv.T

In [ ]:
skew(np.array([1,2,3]))

In [ ]:
Ds = []
for MA, MB in zip(MAs, MBs):
    D = np.zeros((6,8))
#     a = MA.log().unit().to_array().copy()[:3]
#     b = MB.log().unit().to_array().copy()[:3]
#     aprime = MA.log().unit().to_array().copy()[3:]
#     bprime = MB.log().unit().to_array().copy()[3:]
#     print(MA)
    

#     MA = MA.spin(vsr.Trs.from_vector(vsr.Vec(*np.random.randn(3) * 0.001 )) *
#                                       vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
#                                                             np.random.randn() * 0.01))
#     MB = MB.spin(vsr.Trs.from_vector(vsr.Vec(*np.random.randn(3) * 0.001 )) *
#                                       vsr.Rot.from_bivector(vsr.Biv(*np.random.rand(3)).unit() * 
#                                                             np.random.randn() * 0.01))

#     print(MA)

    a = MA.to_array().copy()[1:4]
    a = np.array([a[2], -a[1], a[0]])
    b = MB.to_array().copy()[1:4]
    b = np.array([b[2], -b[1], b[0]])
    aprime = MA.to_array().copy()[4:7]
    bprime = MB.to_array().copy()[4:7]
    # Upper 3
    D[:3,0] = a - b
    D[:3,1:4] = skew(a + b)
    # Lower 3
    D[3:,0] = aprime - bprime
    D[3:,1:4] = skew(aprime + bprime)
    D[3:,4] = a - b 
    D[3:,5:8] = skew(a + b)
    Ds.append(D)
Ds = np.array(Ds).reshape(-1,8)

In [ ]:
Ds = []
for LA, LB in zip(LAs, LBs):
    LA = LA.unit()
    LB = LB.unit()
    D = np.zeros((6,8))
    a = LA.to_array().copy()[:3]
    a = np.array([a[2], -a[1], a[0]])
    b = LB.to_array().copy()[:3]
    b = np.array([b[2], -b[1], b[0]])
    aprime = LA.to_array().copy()[3:]
    bprime = LB.to_array().copy()[3:]
    # Upper 3
    D[:3,0] = a - b
    D[:3,1:4] = skew(a + b)
    # Lower 3
    D[3:,0] = aprime - bprime
    D[3:,1:4] = skew(aprime + bprime)
    D[3:,4] = a - b 
    D[3:,5:8] = skew(a + b)  
    Ds.append(D)
Ds = np.array(Ds).reshape(-1,8)

In [ ]:
[U, s, Vt] = np.linalg.svd(Ds)
print(Vt.T)
v7 = Vt.T[:,-2]
v8 = Vt.T[:,-1]
u1 = v7[:4]
v1 = v7[4:]
u2 = v8[:4]
v2 = v8[4:]

In [ ]:
a = np.inner(u1,v1)
b = np.inner(u1,v2) + np.inner(u2,v1)
c = np.inner(u2,v2)
[s1, s2] = np.roots([a,b,c])
print(s1, s2)

In [ ]:
val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

if val1 > val2:
    s = s1
    val = val1
else:
    s = s2
    val = val2

lambda2 = np.sqrt(1./val)
lambda1 = s * lambda2
print(lambda1)
print(lambda2)

In [ ]:
m_arr = lambda1 * v7 + lambda2 * v8
print(M_eye_in_hand.to_array())
# m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
# m_arr[4:8] = np.roll(m_arr[4:8], -1)
print(m_arr)

m = vsr.Mot(*m_arr)
print(m * m.rev())

In [ ]:
M_eye_in_hand * M_eye_in_hand.rev()

In [ ]:
def polar_decomposition(x):
    xx = x * x.rev()
    sinv = (vsr.Sca(1.0) + (vsr.Drt(- xx[7] / (2 * xx[0]))))
    return x * sinv

In [ ]:
m2 = polar_decomposition(m)
print(m2)

In [ ]:
M_eye_in_hand

In [ ]:
(np.sqrt(1./val2) * s2) * v7 + np.sqrt(1./val2) * v8

In [ ]:
(np.sqrt(1./val1) * s1) * v7 + np.sqrt(1./val1) * v8

In [ ]:
M_eye_in_hand

In [ ]:
np.inner(M_eye_in_hand.to_array()[:4], M_eye_in_hand.to_array()[:4])